In [219]:
import requests
from bs4 import BeautifulSoup
import tokenize
from io import StringIO
import builtins

keywords = ['False', 'None', 'True', 'and', 'as', 'assert', 'break', 'class', 'continue', 'def', 'del', 'elif', 'else', 'except', 'finally', 'for', 'from', 'global', 'if', 'import', 'in', 'is', 'lambda', 'nonlocal', 'not', 'or', 'pass', 'raise', 'return', 'try', 'while', 'with', 'yield']
keywords = keywords+dir(builtins)
def get_maintext(text1):
    newlis = []

    comment_flag = False

    maintext = []

    for i in text1:
        str1 = []
        if(i.startswith('+') and not i.startswith('+++')):
            if(not i[1:].strip().startswith("#") and not i[1:].strip().startswith("//")):
                if("/*" in i[1:].strip() or "\"\"\"" in i[1:].strip()):
                    comment_flag = True
                if("*/" in  i[1:].strip() or "\"\"\"" in i[1:].strip()):
                    comment_flag = False
                    continue
                if(comment_flag):
                    continue
                tempor = i[1:].strip()
                try:
                    g = tokenize.generate_tokens(StringIO(tempor).readline)
                    for token in g:
                        if(token[1] in keywords):
                            str1.append(token[1])
                        elif('ENDMARKER' not in str(tokenize.tok_name[token[0]]) or 'NEWLINE' not in str(tokenize.tok_name[token[0]])):
                            str1.append(tokenize.tok_name[token[0]])
                except:
                    continue
        if(str1!=[]):
            maintext.append(" ".join(str1))

    return "\n".join(maintext)


url1 = "https://github.com/django/django/pulls"

page1=requests.get(url1)
soup1 = BeautifulSoup(page1.text, 'html.parser')
d = soup1.findAll(class_ = "link-gray-dark")

lis1 = []
lis2 = []

pr_tokenized = []

for i in d:
    if("pull" in i['href']):
        try:
            pull_number = i["id"][6:-5]
            pull_text = i.contents
            lis2.append(pull_text)
            url = "https://patch-diff.githubusercontent.com/raw"+str(i["href"])+".patch"
            page=requests.get(url)
            soup = BeautifulSoup(page.text, 'html.parser')
            text = soup.findAll()
            text1 = str(text).split("\n")
            pr_tokenized.append(get_maintext(text1))
        except:
            print("err")

In [220]:
pr_hashes = []

for i in pr_tokenized:
    pr_hashes.append(hash(i))
final_data = {}

for i in range(len(lis2)):
    final_data[i] = {"text":lis2[i],"hash_value":pr_hashes[i]}


In [221]:
final_list = sorted(final_data.items(), key = lambda x:x[1]['hash_value'])

In [260]:
final_list

[(16, {'text': ['WIP Ticket #29607'], 'hash_value': -8759315563904734428}),
 (15,
  {'text': ['Fixed #30366 -- Fixed StatReloaderTests with HFS+ filesystems'],
   'hash_value': -8114399033693546922}),
 (20,
  {'text': ['Fixed #23521 -- Support changing model bases in migrations'],
   'hash_value': -7954086654390467649}),
 (4,
  {'text': ['Fixed #30388 -- Made inspectdb output OneToOneFields for unique relations.'],
   'hash_value': -7896720591678323225}),
 (18,
  {'text': ['Fixed #23004 -- Hid sensitive environment parameters in debug view'],
   'hash_value': -6587107894054296989}),
 (1,
  {'text': ['[WIP] Fixed #28053 -- Support db_index=Index()'],
   'hash_value': -5800624161158744066}),
 (17,
  {'text': ['Fixed #26803 -- Added --stats-only argument to makemessages'],
   'hash_value': -4901767351062729637}),
 (22,
  {'text': ['Use the cookie signer to sign message cookies.'],
   'hash_value': -4824556446915769619}),
 (14,
  {'text': ['#25633: Updated Spatialite installation instructi

In [259]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

count_vectorizer = CountVectorizer()

sparse_matrix = count_vectorizer.fit_transform(pr_tokenized)

doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix)

from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(df, df))

[[1.         0.99384027 0.96617718 0.         0.95247885 0.93999449
  0.99364404 0.98773996 0.85852735 0.63484918 0.98475338 0.9942417
  0.7571882  0.99788668 0.95960057 0.97253958 0.96387275 0.98720909
  0.98932162 0.99366244 0.99563361 0.         0.98736892 0.99629849
  0.9802379 ]
 [0.99384027 1.         0.96053345 0.         0.94962443 0.91569388
  0.98876119 0.97422141 0.84155221 0.60577487 0.97991099 0.99006086
  0.72844695 0.98926012 0.95640749 0.96770034 0.96206151 0.97428761
  0.99753247 0.98139447 0.9901767  0.         0.9755387  0.98835248
  0.95957713]
 [0.96617718 0.96053345 1.         0.         0.87748978 0.97666574
  0.98554484 0.97694439 0.95034933 0.77256333 0.9693337  0.98423343
  0.87881002 0.97180738 0.99345619 0.92277008 0.99589471 0.98060038
  0.94656904 0.96874935 0.96672656 0.         0.98654454 0.9727161
  0.96927489]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         